## Deep Learning

In [50]:
import os
import pandas as pd
import math
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import History
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import History
from keras.callbacks import EarlyStopping, ModelCheckpoint
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns  #Seaborn is another powerful visulization library for Python
from sklearn.linear_model import Ridge
import os

In [51]:
path = "C:\\Users\jmatney\Documents\GitHub\IndianaRisk\data"
df = pd.read_csv(os.path.join(path, "IndianaRisk.csv"))

In [52]:
df_data = df.loc[:, df.columns != 'subwatershed']

In [54]:
#Define a variable for each type of feature
target = ["claims_total_building_insurance_coverage_avg"]
columns = [x for x in df if x != "claims_total_building_insurance_coverage_avg"]
numeric_columns = [x for x in columns if x != "subwatershed"]

In [57]:
# Normalise
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_data)

In [66]:
from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
# Fixed dimensions
input_dim = df_data.shape[1]  # 8
encoding_dim = 3
# Number of neurons in each Layer [8, 6, 4, 3, ...] of encoders
input_layer = Input(shape=(input_dim, ))
encoder_layer_1 = Dense(6, activation="tanh", activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder_layer_2 = Dense(4, activation="tanh")(encoder_layer_1)
encoder_layer_3 = Dense(encoding_dim, activation="tanh")(encoder_layer_2)

In [67]:
# Crear encoder model
encoder = Model(inputs=input_layer, outputs=encoder_layer_3)
# Use the model to predict the factors which sum up the information of interest rates.
encoded_data = pd.DataFrame(encoder.predict(data_scaled))
encoded_data.columns = ['factor_1', 'factor_2', 'factor_3']

In [68]:
encoded_data

,factor_1,factor_2,factor_3
0,0.125464,-0.218238,-0.056406
1,0.344653,-0.174668,-0.030308
2,0.112020,-0.242349,-0.158765
3,0.129562,-0.140144,-0.051501
4,0.020781,0.020766,0.214785
...,...,...,...
95,0.523242,-0.192554,-0.172746
96,0.135218,-0.097426,-0.103977
97,0.320945,0.054176,-0.123980
98,0.407396,-0.264483,-0.263009


## Min Max Scaler

In [55]:
X = df_data.loc[:, df_data.columns != 'claims_total_building_insurance_coverage_avg']
y = df_data.loc[:, df_data.columns == 'claims_total_building_insurance_coverage_avg']

In [19]:
scaler = MinMaxScaler()

# create target scaler object
scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
scalarX.fit(X)
scalarY.fit(y)

MinMaxScaler()

## Train Test Split

In [24]:
#Create train and test dataset with an 80:20 split
x_train, x_test, y_train, y_test = train_test_split(df_data[numeric_columns],df_data[target],test_size=0.2,random_state=2018)
# Further divide training dataset into train and validation dataset with an 90:10 split
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=2018)

# transform target variables
y_train = scalarY.transform(y_train)
y_test = scalarY.transform(y_test)
y_val = scalarY.transform(y_val)

x_train = scalarX.transform(x_train)
x_test = scalarX.transform(x_test)
x_val = scalarX.transform(x_val)

#Check the sizes of all newly created datasets
print("Shape of x_train:",x_train.shape)
print("Shape of x_val:",x_val.shape)
print("Shape of x_test:",x_test.shape)
print("Shape of y_train:",y_train.shape)
print("Shape of y_val:",y_val.shape)
print("Shape of y_test:",y_test.shape)

Shape of x_train: (72, 40)
Shape of x_val: (8, 40)
Shape of x_test: (20, 40)
Shape of y_train: (72, 1)
Shape of y_val: (8, 1)
Shape of y_test: (20, 1)


In [33]:

# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=4)]

model4 = Sequential()
model4.add(Dense(350,input_dim = 40,activation="relu"))
model4.add(Dropout(0.2, input_shape=(40,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(40,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(40,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(40,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(40,)))
model4.add(Dense(1,activation = "linear"))

model4.compile(optimizer='adam',loss="mean_squared_error",
               metrics=[tf.keras.metrics.RootMeanSquaredError()])

model4.fit(x_train,y_train, 
           validation_data=(x_val,y_val), 
           epochs=450,
           batch_size=64)
#,callbacks=callbacks)

result = model4.evaluate(x_test,y_test)

for i in range(len(model4.metrics_names)):
    print("Metric ",model4.metrics_names[i],":",str(round(result[i],2)))
    


NameError: name 'Sequential' is not defined